In [38]:
from datetime import datetime
from datetime import timedelta
from sportsreference.ncaab.boxscore import Boxscores
from sportsreference.ncaab.teams import Teams
import pandas as pd
import csv
import os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from csv import reader
from csv import writer
import time
path = os.getcwd()
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import cross_val_score
from statistics import mean
import numpy as np

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
kp = pd.read_csv('all.csv')
er = pd.read_csv('all_e.csv')
er.drop(['away','home_score','away_score','score'], axis = 1, inplace = True)
df = pd.merge(er, kp, how='left', left_on=['home','date'], right_on=['home','date'])
df

,date,draft_kings,home,lasso,linear,predict_kp,rf,adj_def_x,adj_def_y,adj_off_x,adj_off_y,adj_tempo_x,adj_tempo_y,away,away_score,conf_x,conf_y,experience_x,experience_y,fg_def_x,fg_def_y,fg_off_x,fg_off_y,height_x,height_y,home_score,poss_def_x,poss_def_y,poss_off_x,poss_off_y,reb_def_x,reb_def_y,reb_off_x,reb_off_y,score,three_def_x,three_def_y,three_off_x,three_off_y,turnover_def_x,turnover_def_y,turnover_off_x,turnover_off_y,wins_five_x,wins_five_y,wins_last_x,wins_last_y
0,2021-01-12,3.5,Michigan,-7.0,-8.0,1,-16.0,91.4,89.2,116.4,114.9,68.0,64.9,Wisconsin,54.0,B10,B10,2.10,2.33,43.8,44.4,59.0,53.8,77.9,78.2,77.0,17.6,18.0,16.6,18.1,24.1,21.9,31.1,23.7,23.0,34.8,33.0,35.8,41.4,15.2,18.2,18.2,12.1,5.0,4.0,1.0,1.0
1,2021-01-12,-3.5,Oklahoma St.,-3.0,-2.0,-1,-4.0,92.3,89.5,109.8,112.7,71.7,68.7,Kansas,70.0,B12,B12,1.12,1.42,46.8,47.2,51.3,49.9,77.6,78.2,75.0,17.6,17.5,15.7,16.3,29.6,25.9,32.2,35.0,5.0,33.2,32.1,31.6,35.5,19.5,18.1,19.2,18.0,2.0,4.0,1.0,1.0
2,2021-01-12,-1.0,Virginia Tech,-6.0,-2.0,1,-6.0,93.9,92.9,111.7,112.8,67.2,69.0,Duke,67.0,ACC,ACC,1.63,0.90,48.8,52.2,53.0,52.7,76.3,76.9,74.0,18.0,16.8,17.7,16.4,23.0,24.9,31.8,35.3,7.0,33.8,37.1,35.3,35.2,18.6,21.8,17.9,20.0,4.0,4.0,1.0,1.0
3,2021-01-12,3.0,Kentucky,1.0,2.0,1,-4.0,91.1,93.0,107.7,111.6,68.0,74.6,Alabama,85.0,SEC,SEC,0.90,1.93,45.9,47.4,47.2,49.4,79.0,77.9,65.0,18.0,17.5,17.0,14.2,30.4,29.2,33.8,32.8,-20.0,30.8,31.6,31.1,32.2,18.3,21.0,20.8,18.0,3.0,5.0,1.0,1.0
4,2021-01-12,4.5,North Carolina,6.0,4.0,3,-4.0,90.3,92.7,108.6,109.6,71.1,70.6,Syracuse,75.0,ACC,ACC,1.08,1.49,48.7,46.3,46.1,51.3,78.8,77.9,81.0,17.7,17.0,15.8,15.3,23.7,28.0,40.7,29.6,6.0,34.7,29.8,30.4,32.6,18.8,22.8,22.0,17.5,3.0,4.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,2021-03-04,8.0,Grambling St.,8.0,7.0,9,7.0,105.3,108.8,90.4,85.0,70.4,71.1,Alabama St.,68.0,SWAC,SWAC,2.28,1.81,46.9,47.1,46.9,43.8,77.4,75.2,91.0,16.8,17.0,16.6,15.8,31.7,29.8,24.8,30.0,23.0,34.2,30.1,32.2,31.9,21.0,21.3,21.8,22.0,2.0,1.0,0.0,0.0
1856,2021-03-04,12.0,Texas Southern,10.0,14.0,10,11.0,105.4,110.3,98.8,92.0,71.7,69.0,Alcorn St.,78.0,SWAC,SWAC,2.54,2.19,47.2,56.0,48.0,44.3,77.8,74.9,80.0,16.2,16.7,16.2,17.4,26.0,31.0,32.0,30.1,2.0,33.3,36.6,29.0,29.6,19.2,22.4,21.4,19.9,4.0,3.0,1.0,0.0
1857,2021-03-04,-11.0,Delaware St.,-9.0,-10.0,-11,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1858,2021-03-04,13.5,Santa Clara,13.0,16.0,13,11.0,96.5,110.3,98.4,94.6,70.7,71.7,Portland,86.0,WCC,WCC,1.94,1.99,45.9,58.4,45.8,45.3,79.0,75.9,95.0,17.8,16.0,15.6,17.0,25.0,30.7,22.9,17.6,9.0,30.5,37.1,28.9,32.5,16.8,17.8,18.4,19.2,0.0,0.0,0.0,0.0


In [10]:
df.dropna(subset=['score'], inplace = True)

In [13]:
df['result'] = df['score'].apply(lambda x: 1 if x>0 else 0)

In [14]:
df

,date,draft_kings,home,lasso,linear,predict_kp,rf,adj_def_x,adj_def_y,adj_off_x,adj_off_y,adj_tempo_x,adj_tempo_y,away,away_score,conf_x,conf_y,experience_x,experience_y,fg_def_x,fg_def_y,fg_off_x,fg_off_y,height_x,height_y,home_score,poss_def_x,poss_def_y,poss_off_x,poss_off_y,reb_def_x,reb_def_y,reb_off_x,reb_off_y,score,three_def_x,three_def_y,three_off_x,three_off_y,turnover_def_x,turnover_def_y,turnover_off_x,turnover_off_y,wins_five_x,wins_five_y,wins_last_x,wins_last_y,result
0,2021-01-12,3.5,Michigan,-7.0,-8.0,1,-16.0,91.4,89.2,116.4,114.9,68.0,64.9,Wisconsin,54.0,B10,B10,2.10,2.33,43.8,44.4,59.0,53.8,77.9,78.2,77.0,17.6,18.0,16.6,18.1,24.1,21.9,31.1,23.7,23.0,34.8,33.0,35.8,41.4,15.2,18.2,18.2,12.1,5.0,4.0,1.0,1.0,1
1,2021-01-12,-3.5,Oklahoma St.,-3.0,-2.0,-1,-4.0,92.3,89.5,109.8,112.7,71.7,68.7,Kansas,70.0,B12,B12,1.12,1.42,46.8,47.2,51.3,49.9,77.6,78.2,75.0,17.6,17.5,15.7,16.3,29.6,25.9,32.2,35.0,5.0,33.2,32.1,31.6,35.5,19.5,18.1,19.2,18.0,2.0,4.0,1.0,1.0,1
2,2021-01-12,-1.0,Virginia Tech,-6.0,-2.0,1,-6.0,93.9,92.9,111.7,112.8,67.2,69.0,Duke,67.0,ACC,ACC,1.63,0.90,48.8,52.2,53.0,52.7,76.3,76.9,74.0,18.0,16.8,17.7,16.4,23.0,24.9,31.8,35.3,7.0,33.8,37.1,35.3,35.2,18.6,21.8,17.9,20.0,4.0,4.0,1.0,1.0,1
3,2021-01-12,3.0,Kentucky,1.0,2.0,1,-4.0,91.1,93.0,107.7,111.6,68.0,74.6,Alabama,85.0,SEC,SEC,0.90,1.93,45.9,47.4,47.2,49.4,79.0,77.9,65.0,18.0,17.5,17.0,14.2,30.4,29.2,33.8,32.8,-20.0,30.8,31.6,31.1,32.2,18.3,21.0,20.8,18.0,3.0,5.0,1.0,1.0,0
4,2021-01-12,4.5,North Carolina,6.0,4.0,3,-4.0,90.3,92.7,108.6,109.6,71.1,70.6,Syracuse,75.0,ACC,ACC,1.08,1.49,48.7,46.3,46.1,51.3,78.8,77.9,81.0,17.7,17.0,15.8,15.3,23.7,28.0,40.7,29.6,6.0,34.7,29.8,30.4,32.6,18.8,22.8,22.0,17.5,3.0,4.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1854,2021-03-04,8.0,Jackson St.,8.0,7.0,9,9.0,99.0,105.8,88.4,86.3,69.6,68.2,Alabama A&M,35.0,SWAC,SWAC,2.47,0.73,44.6,44.5,43.0,45.1,76.6,76.5,50.0,16.8,16.5,16.7,17.6,26.7,29.4,29.4,28.9,15.0,32.7,27.5,31.5,28.3,21.7,18.0,22.4,21.3,5.0,0.0,1.0,0.0,1
1855,2021-03-04,8.0,Grambling St.,8.0,7.0,9,7.0,105.3,108.8,90.4,85.0,70.4,71.1,Alabama St.,68.0,SWAC,SWAC,2.28,1.81,46.9,47.1,46.9,43.8,77.4,75.2,91.0,16.8,17.0,16.6,15.8,31.7,29.8,24.8,30.0,23.0,34.2,30.1,32.2,31.9,21.0,21.3,21.8,22.0,2.0,1.0,0.0,0.0,1
1856,2021-03-04,12.0,Texas Southern,10.0,14.0,10,11.0,105.4,110.3,98.8,92.0,71.7,69.0,Alcorn St.,78.0,SWAC,SWAC,2.54,2.19,47.2,56.0,48.0,44.3,77.8,74.9,80.0,16.2,16.7,16.2,17.4,26.0,31.0,32.0,30.1,2.0,33.3,36.6,29.0,29.6,19.2,22.4,21.4,19.9,4.0,3.0,1.0,0.0,1
1858,2021-03-04,13.5,Santa Clara,13.0,16.0,13,11.0,96.5,110.3,98.4,94.6,70.7,71.7,Portland,86.0,WCC,WCC,1.94,1.99,45.9,58.4,45.8,45.3,79.0,75.9,95.0,17.8,16.0,15.6,17.0,25.0,30.7,22.9,17.6,9.0,30.5,37.1,28.9,32.5,16.8,17.8,18.4,19.2,0.0,0.0,0.0,0.0,1


In [15]:
x = df[['adj_off_x', 'adj_def_x','adj_tempo_x',
       'adj_off_y', 'adj_def_y','adj_tempo_y']].values
y = df[['result']].values

In [17]:
clf = RandomForestClassifier(n_estimators=10000)
print(mean(cross_val_score(clf, x, y, cv=10)))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Prog

0.6812615101289135


In [23]:
clf = LogisticRegression()
print(mean(cross_val_score(clf, x, y, cv=10)))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\va

0.6956537753222836


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [24]:
clf.fit(x,y)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [25]:
final = pd.read_csv('final.csv')

In [26]:
final

,teams,conf,adj_off,adj_def,adj_tempo,poss_off,poss_def,fg_off,fg_def,turnover_off,turnover_def,reb_off,reb_def,three_off,three_def,experience,height,wins_five,wins_last
0,Gonzaga,WCC,126.8,88.8,74.8,14.1,17.8,61.0,47.5,16.1,20.3,30.4,23.4,36.5,32.5,1.53,78.1,0,0
1,Michigan,B10,120.1,87.9,66.8,17.3,18.3,54.9,44.9,16.3,15.1,29.4,24.8,38.7,33.5,2.14,77.9,0,0
2,Illinois,B10,119.7,87.6,70.7,16.8,17.0,55.6,46.6,18.2,16.0,33.0,22.2,37.6,32.7,1.64,77.0,0,0
3,Baylor,B12,124.0,93.0,68.4,16.6,17.5,57.5,49.1,17.6,24.6,37.5,31.0,41.8,34.0,2.00,76.6,0,0
4,Iowa,B10,124.2,93.9,70.0,16.2,17.4,54.6,48.3,13.4,16.3,30.7,28.6,38.6,34.8,1.71,77.9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,South Carolina St.,MEAC,87.7,111.2,72.0,16.6,16.4,44.9,55.6,23.9,22.6,26.2,33.4,31.9,36.1,1.00,76.6,0,0
353,Alabama St.,SWAC,84.6,109.7,71.1,16.0,16.8,43.5,48.9,22.3,20.6,29.5,29.6,31.7,32.5,1.82,75.2,0,0
354,Maryland Eastern Shore,MEAC,84.7,110.4,68.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,100.0,0.0,100.0,0.00,0.0,0,0
355,Chicago St.,WAC,89.0,116.9,67.8,17.8,16.0,41.3,60.1,21.7,16.2,16.6,37.9,25.1,39.5,1.88,75.6,0,0


In [109]:
home = final[final['teams'] =='Illinois']
home_values = home[['adj_off', 'adj_def','adj_tempo']].values
away = final[final['teams'] =='Loyola Chicago']
away_values = away[['adj_off', 'adj_def','adj_tempo']].values

In [110]:
away_values

array([[111.1,  85.9,  64.2]])

In [111]:
home_values

array([[119.7,  87.6,  70.7]])

In [112]:
test  = np.concatenate((home_values, away_values), axis=None)

In [113]:
test 

array([119.7,  87.6,  70.7, 111.1,  85.9,  64.2])

In [114]:
y_pred_prob = clf.predict_proba([test])

In [115]:
y_pred_prob[0][1]

0.8075943552139688

In [116]:
y_pred = clf.predict([test])

In [117]:
y_pred

array([1], dtype=int64)

In [118]:
test  = np.concatenate((away_values, home_values), axis=None)

In [119]:
y_pred_prob = clf.predict_proba([test])
y_pred_prob

array([[0.53093476, 0.46906524]])

In [120]:
y_pred = clf.predict([test])
y_pred

array([0], dtype=int64)

In [121]:
from random import random

In [131]:
higher_seed = input('Team A: ')
lower_seed = input('Team B: ')
home = final[final['teams'] == higher_seed]
home_values = home[['adj_off', 'adj_def','adj_tempo']].values
away = final[final['teams'] == lower_seed]
away_values = away[['adj_off', 'adj_def','adj_tempo']].values

test  = np.concatenate((home_values, away_values), axis=None)
y_pred_prob = clf.predict_proba([test])
game_1 = y_pred_prob[0][1]


test  = np.concatenate((away_values, home_values), axis=None)
y_pred_prob = clf.predict_proba([test])
game_2 = y_pred_prob[0][0]


mean = (game_1 + game_2)/2
print(str(round(mean,2) * 100)+' percent chance '+ higher_seed+' wins')

random_val = random()

if random_val <= mean:
    print(higher_seed+' wins')
else:
    print(lower_seed+' wins')

Team A: USC
Team B: Drake
67.0 percent chance USC wins
USC wins
